In [14]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pyarrow as pa
import matplotlib.pyplot as plt
import os
import glob

In [15]:
#Como se puede observar los estados estan dividido en varios json por lo cual vamos a proceder a unrilos en uno solo


# Directorio donde están los archivos JSON
directorio = ""

# Cargar todos los archivos JSON en una lista de DataFrames
dataframes = []
for filename in sorted(glob.glob(os.path.join(directorio, "*.json"))):
    df = pd.read_json(filename, lines= True)
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df = pd.concat(dataframes, ignore_index=True)


**CONSERVAMOS SOLO LOS RESTAURANTES**

In [16]:
df['category'] = df['category'].astype(str)

In [17]:
df = df[df['category'].str.contains("restaurant", case=False, na=False)]

In [18]:
df.reset_index(drop=True, inplace=True)

**ELIMINACION DE COLUMNAS QUE NO NOS SIRVEN**

In [19]:
df.drop('description', axis = 1, inplace=True)
df.drop('state', axis = 1, inplace = True)
df.drop('relative_results', axis= 1, inplace = True)
df.drop('url', axis = 1, inplace= True)
df.drop('address', axis = 1, inplace = True)

**NORMALIZAMOS LOS DATOS**

In [20]:
df['price'].value_counts()

price
$       57995
$$      33222
₩₩       3579
₩        3438
$$$      1390
$$$$      180
₩₩₩       160
₩₩₩₩       36
Name: count, dtype: int64

In [21]:
df['price'] = df['price'].replace({'₩': '$', '₩₩': '$$', '₩₩₩': '$$$', '₩₩₩₩': '$$$$'})

**TRATAMOS NULOS**

In [45]:
df.isnull().sum()

name                   0
gmap_id                0
latitude               0
longitude              0
category               0
avg_rating             0
num_of_reviews         0
price             111215
hours              21940
MISC                1404
dtype: int64

In [23]:
df['price'] = df['price'].replace({None: np.nan, '': np.nan})
df['hours'] = df['hours'].replace({None: np.nan, '': np.nan})
df['MISC'] = df['MISC'].replace({None: np.nan, '': np.nan})

**TRATAMIENTO DE DUPLICADOS**

In [44]:
df = df.drop_duplicates(subset='gmap_id', keep='first')

In [25]:
df.to_json("metadata_sitios_limpio.json", orient="records", lines=True)

In [46]:
df['gmap_id'].to_json('gmap_id_restaurantes.json')